In [ ]:
from src.hybrid_search import hybrid_search, rrf_fusion
from src.reranker import rerank_results
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchAny
from importlib import reload
import src.config

### SEARCH QUERY ###
query = "romantic robots taking over the sea"
user_ids = [1, 5, 8]  # 🔁 Accepts multiple user_ids
collection_name = config["collection_name"]


# Reload the config module to ensure the latest config.yaml is used
reload(src.config)
config = src.config.load_config()

# Initialize Qdrant client
_qdrant = QdrantClient(
    host=config["qdrant"]["host"],
    port=config["qdrant"]["port"],
    timeout=config["qdrant"].get("timeout", 60.0)
)

# Run hybrid search with multi-user filter
dense_results, sparse_results = hybrid_search(query, user_ids=user_ids)
fused = rrf_fusion([dense_results, sparse_results])[:10]

# Rerank top fused results
docs = [
    hit.payload["title"] + " " + hit.payload.get("overview", "")
    for hit in fused
]
scores = rerank_results(query, docs)

# Display results
print(f"🔍 Search for user(s) {user_ids}")
print(f"❓ Movies about {query}:\n")

for i, (hit, score) in enumerate(zip(fused, scores), 1):
    print(f"{i}. {hit.payload['title']} | Score: {score:.4f}")
    
# Get total count of records in the collection
total_count = _qdrant.count(collection_name=collection_name).count
print(f"\n📊 Total records in the collection: {total_count}")

# Get count of records filtered by user_ids
query_filter = Filter(
    must=[
        FieldCondition(
            key="user_id",
            match=MatchAny(any=user_ids)
        )
    ]
)

# Count matching points
filtered_count = _qdrant.count(
    collection_name=collection_name,
    count_filter=query_filter,
    exact=True
).count

print(f"\n📊 Records matching user_ids {user_ids}: {filtered_count}")

🔍 Search for user(s) [1, 5, 8]
❓ Movies about) romantic robots taking over the sea

1. Pure Emotions of the Sea | Score: -1.3433
2. The Automatic Motorist | Score: -0.3948
3. Robots | Score: -1.7817
4. Cosmic Fairy Tale | Score: -1.1875
5. Lover and the Sea | Score: -2.0970
6. Many Waters | Score: -3.0186
7. White Acacia | Score: -2.5272
8. Ocean | Score: -3.0041
9. The Adventure of Goar | Score: -2.0781
10. Tsunami | Score: -3.5968

📊 Total records in the collection: 1196770

📊 Records matching user_ids [1, 5, 8]: 358933
